In [1]:
!pip install requests beautifulsoup4 openai langchain-openai


  Using cached openai-2.15.0-py3-none-any.whl.metadata (29 kB)
  Using cached langchain_openai-1.1.7-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_core-1.2.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached langsmith-0.6.2-py3-none-any.whl.metadata (15 kB)
  Using cached uuid_utils-0.13.0-cp39-abi3-win_amd64.whl.metadata (5.5 kB)
Using cached openai-2.15.0-py3-none-any.whl (1.1 MB)
Using cached langchain_openai-1.1.7-py3-none-any.whl (84 kB)
Using cached langchain_core-1.2.7-py3-none-any.whl (490 kB)
Using cached langsmith-0.6.2-py3-none-any.whl (282 kB)
   ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
   ----------------------------------- ---- 786.4/879.1 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 879.1/879.1 kB 4.4 MB/s eta 0:00:00
Using cached uuid_utils-0.13.0-cp39-abi3-win_amd64.whl (183 kB)

   ----- ---------------------------------- 1/8 [orjson]
   -------------------- ------------------- 4/8 [openai]
   -----------

In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
      text = soup.get_text(separator= ' ')
#limpar texto
      lines = (line.strip() for line in text.splitlines())
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      texto_limpo = '\n'.join(chunk for chunk in chunks if chunk)
      return texto_limpo

    else:
      print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
      return None

    text = soup.get_text()
    return text
extract_text_from_url('https://dev.to/alextheluchador/how-i-passed-the-azure-ai-102-exam-4gg7')

In [ ]:
from langchain_openai.chat_models.azure import AzureChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    azure_api_key=os.getenv("AZURE_API_KEY"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    model="gpt-4o",
    max_retries=0
)

def translate_text(text, target_language="Portuguese"):
    prompt = f"Translate the following text to {target_language}:\n\n{text}"
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that translates text."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message['content']

In [ ]:
extracted_text = extract_text_from_url("<URL OCULTADA>")
article = translate_text(extracted_text, target_language="Portuguese")

print(article)